# Final notebook for Neuropep project V1
* To get candidate peptides
* To generate good-looking ifgures
* For debugging

This is for the V1 version of the program, meaning it is BAREBONES but functional.

I'm mainly using this because I'm an exploratory programmer and this helps me find out exactly the methods and sequences I need to use to implement what is left to be implemented

Here's the Class for the sequence similarity calculator:

In [11]:
import pandas as pd
import numpy as np
from scipy.stats import kendalltau
import matplotlib.pyplot as plt
from typing import Set, Tuple, Dict

class SequenceSimilarity:
    '''
    Class that takes in a path to a list of amino acid sequences as well
    as any number of peptide sequences explicitly that are known to have
    a certain set of properties. Generates metrics for similarity for each
    peptide in path and returns domains AA sequence with high similarity
    '''

    def __init__(self, binders: Dict, data_paths: Dict, peps_path: str, aa_col: str):
        
        self.binder_dict = binders
        self.data_paths_dict = data_paths
        self.__update_binders()
        self.__update_similarity_data()

        self.peps = pd.read_csv(peps_path)
        self.aa_col = aa_col

        self.AA = set('LINGVEPHKAYWQMSCTFRD')
        self.sseq_set: set[Tuple[int, str]] # full set of binder subseqs
        self.top_sseq: set[Tuple[int, str]] # set of sub sequences w/ high simil
        self.top_seq: set[str]              # set of peptides with high simil
    
    def __update_binders(self) -> None:
        '''
        Private method to set the binders stored by
        class and check to make sure they are of the
        same length as one another
        '''
        self.binders = [binder for binder in self.binder_dict.values()]
        try:
            self.binder_len = len(self.binders[0])
        except:
            print("Need at least one input binder")
        
        for binder in self.binders:
            if len(binder) != self.binder_len:
                print("All binders must be of same length")
                # @TODO Handle multiple lengths of binders, each differently
                #       lengthed binder is compared with different parts of the
                #       full peptide set of the same length

    def __update_similarity_data(self) -> None:
        """
        Private method to store the paths of any data needed
        for similarity calcs and create Dataframes from them
        """
        self.data_paths: Dict = self.data_paths_dict
        self.data = {data:pd.read_csv(self.data_paths[data], header=None) \
                     for data in self.data_paths.keys()}


    def df_filter_subseq(self, data: pd.DataFrame, sub_seq: str, ind: int = None):
        '''
        Takes in a subsequence of equal or lesser length to
        peptides in class peptide dataframe and returns a dataframe
        containing only those peptides containing the sequence
        '''
        if not {*sub_seq}.issubset({*self.AA}):
            raise Exception('Invalid subsequence')
        if ind is None:
            return self.data.filter() #@TODO Implement the filter here

        data_filter = self.data[self.AA_COL].apply(
            lambda s: s[ind:len(sub_seq) == sub_seq])
        return data[data[self.AA_COL].filter(data_filter)]

    def get_sim_matrix(self, seq) -> pd.DataFrame:
        return self.data.filter


    def get_binder_subseq(self) -> pd.DataFrame:
        '''
        Generates all possible subsequences for binders
        provided in class constructor
        '''
        def gen_all_subseq(seq, sub_seq, i):
            if i == len(seq):
                if len(sub_seq) != 0:
                    yield(sub_seq)
                else:
                    gen_all_subseq(seq, sub_seq, sub_seq)
                gen_all_subseq(seq, sub_seq+[seq[i]], i+1)

        sseq = dict.fromkeys(self.binders)
        for binder in self.binders:
            sseq[binder] = [sseq for sseq in list(
                gen_all_subseq(binder, '', 0))]
        return sseq

    def get_PAM30_similarity(self) -> pd.DataFrame:
        '''
        Returns the PAM30 similarity of peptides to
        specified binder sequences
        @TODO: Automatically get perfect match and lowest match
        @TODO: Generalize for all binder sequences inputted
        '''
        raise NotImplementedError

    def get_BLOSUM_similarity(self) -> pd.DataFrame:
        raise NotImplementedError
    '''
    def get_RRM_SN_ratio(self):
        get_eiip_seq = lambda pep: list(map(lambda aa: self.AA_EIIP[aa], pep))
        get_dft_from_eiip = lambda eiip: np.fft.rfft(eiip)[1:]
    '''

    def get_kendalltau_corr_map(self) -> Tuple:
        return kendalltau(self.data['AA_MAP'][['Num']], self.data['AA_MAP'][['EIIP']])

In [59]:
# converting the 9_17mers.csv Aaron used to more readable format
# was using the csv Savvy was using -- but has less than half the result for 12 len peptides

tmp = pd.read_csv('aaron/9_17mers.csv', header=None)
tmp.columns = ['Sequences']
tmp.to_csv("src_data/Sequence_data.csv")

,0,1
0,0,FSEEEKEPE
1,1,SVPHFSDED
2,2,VPHFSDEDK
3,3,PHFSDEDKD
4,4,HFSDEDKDP
...,...,...
28910,28910,VMGHFRWDR
28911,28911,MGHFRWDRF
28912,28912,YVMGHFRWDR
28913,28913,VMGHFRWDRF


In [60]:
tmp2 = pd.read_csv('src_data/Sequence_data.csv', header=None)
tmp2

,0,FSEEEKEPE
0,1,SVPHFSDED
1,2,VPHFSDEDK
2,3,PHFSDEDKD
3,4,HFSDEDKDP
4,5,FSDEDKDPE
...,...,...
28909,28910,VMGHFRWDR
28910,28911,MGHFRWDRF
28911,28912,YVMGHFRWDR
28912,28913,VMGHFRWDRF


# Necessary data and variables

Here's the necessary data and variables (put in main.py) to construct a SequenceSimilarity ob

In [53]:
DATA_PATHS = {
        "BLOSUM":"src_data/BLOSUM.csv",
        "PAM30":"src_data/pam30.csv",
        "AA_MAP":"src_data/aa_chart.csv",
}
SEQS = {
        'GRBP5':'IMVTESSDYSSY',
        'M6':'IMVTASSAYDDY'
}
AA_COL = 'Sequences'
PEP_PATH = 'src_data/Sequence_data.csv'
            # Consider using src_data/Organized_NPS.csv
            # the one Savvy was using -- less than half
            # the number of peptides of len 12 though
Similarity = SequenceSimilarity(SEQS, DATA_PATHS, PEP_PATH, AA_COL)

In [54]:
# BLOSUM, PAM30, AA Mapping chart stored in Similarity.data["BLOSUM"], etc.
print(Similarity.data['BLOSUM'].shape, Similarity.data['PAM30'].shape)
Similarity.data['AA_MAP'].head()

(25, 25) (28, 28)


,0,1,2,3
0,AA,Num,EIIP,Function
1,F,0,0.0946,Aromatic
2,Y,0,0.0516,Aromatic
3,W,0,0.0548,Aromatic
4,A,1,0.0373,Hydrophobic


In [57]:
# print the binders of interest
print(Similarity.binders)

# Full list of Neuropetides and sequences displayed by Similarity.peps 
Similarity.peps[Similarity.peps.Sequences.str.len() == Similarity.binder_len]

['IMVTESSDYSSY', 'IMVTASSAYDDY']


,Unnamed: 0,Sequences
13,13,SVPHFSDEDKDP
14,14,VPHFSDEDKDPE
28,28,SVPHFSEEEKEA
29,29,VPHFSEEEKEAE
43,43,SVPHFSDEDKDP
...,...,...
28873,28873,FLRRIRPKLKWD
28874,28874,LRRIRPKLKWDN
28875,28875,RRIRPKLKWDNQ
28903,28903,YGGFLRRQFKVV


In [46]:
# Let's get sequences with only the same amount of amino acid length as input peptides
same_length_peptides = Similarity.peps[len(Similarity.peps.Sequences) == Similarity.binder_len]
same_length_peptides

KeyError: False

In [36]:
Similarity.binder_len

12

In [48]:
Similarity.peps.Sequences[len(Similarity.peps.Sequences) == 12]

KeyError: False